# Proofs and verification

The purpose of this notebook is to verify the key proofs of the paper numerically

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from numpy import exp, eye as I, trace as tr, diag, kron
from numpy.linalg import eigh, eig, inv

import matplotlib.pyplot as plt

%matplotlib notebook
np.set_printoptions(precision=5, linewidth=500, threshold=500, suppress=True)

from model.utils import vec, mat, diag_i
from utils import vector_derivative_numerical, matrix_derivative_numerical, hessian_numerical

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1.

For a cost function 

\begin{align}
\newcommand{\vecc}[1]{\text{vec}(#1)}
\newcommand{\diag}[1]{\text{diag}(#1)}
\newcommand{\mat}[1]{\text{mat}(#1)}
\newcommand{\aand}{\quad \text{and} \quad}
\newcommand{\orr}{\quad \text{or} \quad}
\newcommand{\for}{\; \text{for} \;}
\newcommand{\with}{\quad \text{with} \quad}
\newcommand{\where}{\quad \text{where} \quad}
\newcommand{\iif}{\quad \text{if} \quad}
\newcommand{\SN}{\Sigma_N}
\newcommand{\ST}{\Sigma_T}
\newcommand{\SNi}{\Sigma_N^{-1}}
\newcommand{\STi}{\Sigma_T^{-1}}
\newcommand{\tr}[1]{\text{tr}\big(#1\big)}
\newcommand{\R}{\mathbb{R}}
\xi(F) &=  \tr{(Y - S_N F S_T^\top)^{\top}(Y - S_N F S_T^\top)} 
\\ &\quad\quad\; + \gamma \, \tr{K^{-1}F ^{\top} H^{-2}F}
\end{align}

the minimising value of $F$ is given by

$$
F^\star = H^2 S_N^\top \bar{U} \big( J \circ (\bar{U}^\top Y \bar{V})\big) \bar{V}^\top S_T K
$$

where 

$$
S_T K S_T^\top = \bar{V} \bar{\Lambda}_K \bar{V}^\top, \quad \text{and} \quad S_N H^2 S_N^\top = \bar{U} \bar{\Lambda}_H \bar{U}^\top
$$

and $J \in \R^{N' \times T'}$ has the elements given by

$$
J_{ij} = \frac{1}{\bar{\lambda}^K_j \bar{\lambda}^H_i  + \gamma}
$$

In [4]:
# Generate some example data

def get_K(N):
    return np.exp(-(np.linspace(0, 1, N)[:, None] - np.linspace(0, 1, N)[None, :]) ** 2) + 1e-3 * I(N)

T = 6
N = 4

T_ = 5
N_ = 3

ST = np.zeros((T_, T))
ST[range(T_), range(T_)] = 1

SN = np.zeros((N_, N))
SN[range(N_), range(N_)] = 1

sigT =  get_K(T_)
sigN = get_K(N_)

sigTi = inv(sigT)
sigNi = inv(sigN)

K = get_K(T)
Hs = get_K(N)

Ki = inv(K)
Hsi = inv(Hs)

gamma = 0.05

Y = np.random.normal(size=(N_, T_))
F = np.random.normal(size=(N, T))


## 1.1 

The derivative of $\xi$ with respect to $F$ is given by 

$$
\frac{\partial \xi}{\partial F} = 2S_N^\top(S_N  F S_T^\top - Y) S_T + 2\gamma H^{-2} F K^{-1}
$$


In [5]:
def zeta1(F: np.ndarray):
    return tr((Y - SN @ F @ ST.T).T @ (Y - SN @ F @ ST.T)) + gamma * tr(Ki @ F.T @ Hsi @ F)

def deriv1(F: np.ndarray):
    return 2 * SN.T @ (SN @ F @ ST.T - Y) @ ST  + 2 * gamma * Hsi @ F @ Ki

In [6]:
F_ = np.random.normal(size=(N, T))
np.allclose(matrix_derivative_numerical(zeta1, F_), deriv1(F_), atol=1e-5)

True

## 1.2 

The derivative is zero at 

$$
 \vecc{F^\star} = \Big( S_T^\top S_T \otimes S_N^\top S_N + \gamma K^{-1} \otimes H^{-2}\Big)^{-1} \big( S_T^\top \otimes S_N^\top \big) \vecc{Y}
$$



In [7]:
F_star1 = inv(kron(ST.T @ ST, SN.T @ SN) + gamma * kron(Ki, Hsi)) @ kron(ST.T, SN.T) @ vec(Y)
np.allclose(matrix_derivative_numerical(zeta1, mat(F_star1, like=F_)), 0, atol=1e-5)

True

## 1.3 

This can be rewritten as

$$
     \big( K S_T^\top \otimes H^2 S_N^\top \big)  \Big( \gamma I_{T'} \otimes I_{N'} + S_T K S_T^\top \otimes S_N H^2 S_N^\top \Big)^{-1}  \vecc{Y}
$$

In [8]:
F_star2 = kron(K @ ST.T, Hs @ SN.T) @ inv(kron(ST @ K @ ST.T, SN @ Hs @ SN.T) + gamma * kron(I(T_), I(N_))) @ vec(Y)
np.allclose(matrix_derivative_numerical(zeta1, mat(F_star2, like=F_)), 0, atol=1e-5)

True

## 1.4

This can be written as 

$$
 H^2 S_N^\top \bar{U} \big( J \circ (\bar{U}^\top Y \bar{V})\big) \bar{V}^\top S_T K
$$

where 

$$
S_T K S_T^\top = \bar{V} \bar{\Lambda}_K \bar{V}^\top, \quad \text{and} \quad S_N H^2 S_N^\top = \bar{U} \bar{\Lambda}_H \bar{U}^\top
$$

and $J \in \R^{N' \times T'}$ has the elements given by

$$
J_{ij} = \frac{1}{\bar{\lambda}^K_j \bar{\lambda}^H_i  + \gamma}
$$

In [9]:
lamK_, V_ = eigh(ST @ K @ ST.T)
lamH_, U_ = eigh(SN @ Hs @ SN.T)
J = 1 / (np.outer(lamH_, lamK_) + gamma)

F_star3 = Hs @ SN.T @ U_ @ (J * (U_.T @ Y @ V_)) @ V_.T @ ST @ K

np.allclose(matrix_derivative_numerical(zeta1, F_star3), 0, atol=1e-5)

True

# 2. 

$$
\begin{align}
& \xi(F) = \tr{(Y - S_N F S_T^\top)^{\top} \SNi (Y - S_N F S_T^\top)\, \STi} \\
& \quad\quad\quad\quad\quad\quad\quad + \gamma \,\tr{K^{-1}F ^{\top} H^{-2}F}
\end{align}
$$

The minimising value for $\Psi$ can be expressed as

$$
F^\star = B \, (J \circ \bar{Y}) \, C^\top
$$

where

$$
\ST = \Psi \Lambda_{\ST} \Psi^\top, \quad \SN = \Phi \Lambda_{\SN} \Phi^\top
$$

\begin{align*}
\Lambda_{\ST}^{-1/2} \Psi^\top S_T K S_T^\top \Psi  \Lambda_{\ST}^{-1/2} &= \bar{V}  \bar{\Lambda}_K \bar{V}^{\top} \\[0.1cm]
\Lambda_{\SN}^{-1/2} \Phi^\top S_N H^2 S_N^\top \Phi  \Lambda_{\SN}^{-1/2} &= \bar{U} \bar{\Lambda}_H \bar{U}^{\top} 
\end{align*}

and

$$
\begin{align}
J_{ij} &= \frac{1}{\gamma + \bar{\lambda}^K_j \bar{\lambda}^H_i} \\[0.1cm]
B &= H^2 S_N^\top \Phi \Lambda_{\SN}^{-1/2} \bar{U} \\[0.2cm]
C &= K S_T^\top \Psi \Lambda_{\ST}^{-1/2} \bar{V} \\[0.2cm]
\bar{Y} &= \bar{U}^{\top}\Lambda_{\SN}^{-1/2} \Phi^\top Y \Psi \Lambda_{\ST}^{-1/2} \bar{V} \\[0.1cm]
\end{align}
$$

## 1.1

The minimising value of the cost function is 

$$
\vecc{F^\star} = \Big( S_T^\top \STi S_T \otimes S_N^\top \SNi S_N + \gamma K^{-1} \otimes H^{-2}\Big)^{-1} \big( S_T^\top \STi \otimes S_N^\top \SNi \big) \vecc{Y}
$$

In [10]:
def zeta2(F: np.ndarray):
    return tr((Y - SN @ F @ ST.T).T @ sigNi @ (Y - SN @ F @ ST.T) @ sigTi) + gamma * tr(Ki @ F.T @ Hsi @ F)

F_star1 = inv(kron(ST.T @ sigTi @ ST, SN.T @ sigNi @ SN) + gamma * kron(Ki, Hsi)) @ kron(ST.T @ sigTi, SN.T @ sigNi) @ vec(Y)
np.allclose(matrix_derivative_numerical(zeta2, mat(F_star1, like=F_)), 0, atol=1e-5)

True

## 1.2 

This can be expressed as 

$$
\big( C \otimes B\big) \, \text{diag}\big(\vecc{J}\big) \, \vecc{\bar{Y} } 
$$

In [11]:
lamT, Psi = np.linalg.eigh(sigT)
lamN, Phi = np.linalg.eigh(sigN)

PsiLam = Psi * (lamT ** -0.5)
K_ = ST @ K @ ST.T
lamK_, V_ = eigh(PsiLam.T @ K_ @ PsiLam)

PhiLam = Phi * (lamN ** -0.5)
H_ = SN @ Hs @ SN.T
lamH_, U_ = eigh(PhiLam.T @ H_ @ PhiLam)

J = 1 / (np.outer(lamH_, lamK_) + gamma)

B = Hs @ SN.T @ PhiLam @ U_
C = K @ ST.T @ PsiLam @ V_
Y_ = U_.T @ PhiLam.T @ Y @ PsiLam @ V_

In [12]:
F_star2 = B @ (J * Y_) @ C.T
np.allclose(matrix_derivative_numerical(zeta2, F_star2), 0, atol=1e-5)

True

# 3.

Given a fitted GLS KGR model, a lower bound for the marginal variance of the prediction uncertainty for the latent signal $F$ is given by 

$$
\Omega_F = \big(\tilde{U}^{-\top} \circ (H^2 \tilde{U})\big) \, J \, \big(\tilde{V}^{-1}  \circ (\tilde{V}^\top K) \big)
$$

where $\circ$ is the Hadamard product,

$$
S_T^\top \STi S_T K = \tilde{V} \tilde{\Lambda}_K \tilde{V}^{-1}, \quad S_N^\top \SNi S_N H^2 = \tilde{U} \tilde{\Lambda}_H \tilde{U}^{-1}
$$

and 

$$
J_{ij} = \frac{1}{\gamma + \tilde{\lambda}^K_i \tilde{\lambda}^H_j}
$$


## 3.1 

The Hessian is given by 

$$
\Sigma_F^{-1} = S_T^\top \STi S_T \otimes  S_N^\top \SNi S_N  + \gamma K^{-1} \otimes H^{-2}
$$


In [13]:
def zeta3(F):
    F = mat(F, shape=(N, T))
    return 0.5 * (tr((Y - SN @ F @ ST.T).T @ sigNi @ (Y - SN @ F @ ST.T) @ sigTi) + gamma * tr(Ki @ F.T @ Hsi @ F))


def hessian_numerical(f, x):
    
    dx = 0.001
    N = len(x)
    out = np.zeros((N, N))
    
    for i in range(N):
        for j in range(N):
        
            def deriv(g, x, k):
                x_ = x.copy()
                _x = x.copy()
                x_[k] += dx / 2
                _x[k] -= dx / 2
                return (g(x_) - g(_x)) / dx 
                        
            out[i, j] = deriv(lambda y: deriv(f, y, i), x, j)
    
    return out

In [14]:
F_ = np.random.normal(size=(N, T))
hess_analytic = kron(ST.T @ sigTi @ ST, SN.T @ sigNi @ SN ) + gamma * kron(Ki, Hsi)
hess_numeric = hessian_numerical(zeta3, vec(F_))

np.allclose(hess_analytic, hess_numeric)

True

## 3.2

The diagonal of the inverse Hessian is given by 

$$
\Omega_F = \big(\tilde{U}^{-\top} \circ (H^2 \tilde{U})\big) \, J \, \big(\tilde{V}^{-1}  \circ (\tilde{V}^\top K) \big)
$$


In [15]:
lamK_, V_ = eig(ST.T @ sigTi @ ST @ K)
lamH_, U_ = eig(SN.T @ sigNi @ SN @ Hs)
J_ = 1 / (np.outer(lamH_, lamK_) + gamma)
omega = np.real((np.linalg.inv(U_).T * (Hs @ U_)) @ J_ @ (inv(V_) * (V_.T @ K)))

In [16]:
np.allclose(mat(inv(hess_analytic)[range(N * T), range(N * T)], like=F_), omega, atol=1e-5)

True